## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-29-08-51-20 +0000,bbc,"Minister must apologise over Savile claim, say...",https://www.bbc.com/news/articles/cgery3eeqzxo...
1,2025-07-29-08-48-00 +0000,wsj,Stock Market Today: Dollar Extends Rally and D...,https://www.wsj.com/livecoverage/stock-market-...
2,2025-07-29-08-43-59 +0000,bbc,Google launches new AI search feature in UK,https://www.bbc.com/news/articles/clyj4zky4zwo...
3,2025-07-29-08-38-00 +0000,nyt,Ozzy Osbourne Funeral Procession to Be Held in...,https://www.nytimes.com/2025/07/29/arts/music/...
4,2025-07-29-08-37-43 +0000,nypost,Russia kills 21 civilians in Ukraine as the Kr...,https://nypost.com/2025/07/29/world-news/russi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2305/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
41,trump,68
87,deal,20
145,trade,20
90,gaza,18
50,shooting,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
225,2025-07-28-14-42-00 +0000,wsj,Trump said the U.S. reached a trade deal with ...,https://www.wsj.com/economy/trade/european-uni...,139
163,2025-07-28-18-24-16 +0000,nypost,France presses EU to unload ‘trade bazooka’ on...,https://nypost.com/2025/07/28/business/france-...,137
181,2025-07-28-17-17-00 +0000,wsj,President Trump said he would give Russia’s Vl...,https://www.wsj.com/politics/trump-losing-pati...,132
194,2025-07-28-16-27-53 +0000,nypost,Trump hails cease-fire win after halting trade...,https://nypost.com/2025/07/28/us-news/trump-ha...,130
254,2025-07-28-11-56-36 +0000,nyt,"Europe, Worried by Other Global Crises, Agrees...",https://www.nytimes.com/2025/07/28/world/europ...,123


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
225,139,2025-07-28-14-42-00 +0000,wsj,Trump said the U.S. reached a trade deal with ...,https://www.wsj.com/economy/trade/european-uni...
72,76,2025-07-29-00-38-47 +0000,nypost,NYC gunman ID’d as Shane Tamura after deadly s...,https://nypost.com/2025/07/28/us-news/nyc-gunm...
181,64,2025-07-28-17-17-00 +0000,wsj,President Trump said he would give Russia’s Vl...,https://www.wsj.com/politics/trump-losing-pati...
29,51,2025-07-29-06-07-59 +0000,bbc,Police officer among four dead in shooting at ...,https://www.bbc.com/news/articles/cn023751713o...
58,51,2025-07-29-02-14-44 +0000,nyt,"In a First, Leading Israeli Rights Groups Accu...",https://www.nytimes.com/2025/07/28/world/middl...
222,40,2025-07-28-15-04-00 +0000,wsj,Former Gov. Roy Cooper of North Carolina launc...,https://www.wsj.com/politics/elections/roy-coo...
187,40,2025-07-28-16-50-00 +0000,wsj,Business leaders on both sides of the Atlantic...,https://www.wsj.com/economy/trade/companies-we...
8,35,2025-07-29-08-17-56 +0000,nypost,Sen. John Kennedy weighs in on Park Ave skyscr...,https://nypost.com/2025/07/29/us-news/sen-john...
259,33,2025-07-28-11-10-07 +0000,cbc,"Thailand, Cambodia agree to ceasefire after cl...",https://www.cbc.ca/news/world/thailand-cambodi...
69,31,2025-07-29-01-00-00 +0000,wsj,Federal Reserve officials expect they will nee...,https://www.wsj.com/economy/central-banking/a-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
